# Experiment with CIFAR 10
## Centralized learning for baseline

In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import pickle
import time
import datetime

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)
import tensorflow_hub as hub
import numpy as np
import pandas as pd

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
labelnames = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
labelnames

In [ ]:
dataset_root = os.path.abspath(os.path.expanduser('dataset-cifar10'))
dataset_root = os.path.join(dataset_root, 'center')
dataset_root

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
target_size = (224, 224, 3)
datasets = dict()
for subset in ['train', 'test']:
    path = os.path.join(dataset_root, subset)
    datasets[subset] = image_generator.flow_from_directory(path, classes=labelnames,
                                                           target_size=target_size[:-1], shuffle=True,
                                                           follow_links=True)
datasets